In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from pipeline.utils import index_unsorted, nested_insert, nested_select
from pipeline.compact_json_encoder import CompactJSONEncoder
from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr, 
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
    evaluate_decoding
)

from research.data.natural_scenes import (
    NaturalScenesDataset,
    StimulusDataset,
    KeyDataset
)

In [ ]:
nsd_path = Path('D:\\Datasets\\NSD\\')
nsd = NaturalScenesDataset(nsd_path)

subject_name = 'subj01'
betas_params = dict(
    subject_name=subject_name,
    voxel_selection_path='derivatives/voxel-selection.hdf5',
    voxel_selection_key='nc/value',
    threshold=30,
    return_volume_indices=True
)
betas, betas_indices = nsd.load_betas(**betas_params)

model_name = 'ViT-B=32'
stimulus_key = 'embedding'
stimulus_params = dict(
    subject_name=subject_name,
    #stimulus_path='nsddata_stimuli/stimuli/nsd/nsd_stimuli.hdf5',
    #stimulus_key='imgBrick',
    stimulus_path=f'derivatives/stimulus_embeddings/{model_name}.hdf5',
    stimulus_key=stimulus_key,
    delay_loading=True
)
stimulus = nsd.load_stimulus(**stimulus_params)
dataset = KeyDataset({'betas': betas, 'stimulus': stimulus})
train_dataset, val_dataset, test_dataset = nsd.apply_subject_split(dataset, subject_name, 'split-01')

In [ ]:
subject_betas = nsd.subjects[subject_name]['betas']
volume_shape = tuple(subject_betas['betas'].attrs['spatial_shape'])
volume_shape

In [ ]:
train_dataset[0]['stimulus']['data'].shape

In [ ]:
with h5py.File('G:\\Github Repositories\\Google Drive\\Repositories\\neuro-ml\\example_data\\nsd.hdf5', 'w') as f:
    f['betas_indices'] = betas_indices
    f['betas_indices'].attrs['volume_shape'] = volume_shape
    datasets = [('train', val_dataset), ('test', test_dataset)]
    for dset_name, dset in datasets:
        sample = dset[0]
        num_betas = sample['betas'][0].shape[0]
        num_stimulus = sample['stimulus']['data'].shape[0]
        N = len(dset)
        f[f'{dset_name}/betas'] = np.zeros((N, num_betas), dtype='f4')
        f[f'{dset_name}/stimulus'] = np.zeros((N, num_stimulus), dtype='f4')
        
        for i, elem in enumerate(dset):
            f[f'{dset_name}/betas'][i] = elem['betas'][0]
            f[f'{dset_name}/stimulus'][i] = elem['stimulus']['data']
    

In [ ]:
# Load group 1 and 2 results

clip_features_path = Path('D:\\Datasets\\NSD\\derivatives\\stimulus_embeddings\\ViT-B=32-embeddings.hdf5')
clip_features = h5py.File(clip_features_path, 'r')

clip_decoded_features_path = Path('D:\\Datasets\\NSD\\derivatives\\decoded_features\\ViT-B=32')
group1 = h5py.File(clip_decoded_features_path / 'group-1.hdf5', 'r')
group2 = h5py.File(clip_decoded_features_path / 'group-2.hdf5', 'r')

In [ ]:
# Run the standard decoding evaluation

subject = 'subj01'
embedding_names = [*(f'transformer.resblocks.{i}' for i in range(12)), 'embedding']
results = {}
for group_name, group in [('contrastive', group1), ('distance', group2)]:
    for embedding_name in tqdm(embedding_names):
        for fold_name in ('val_averaged', 'val', 'test_averaged', 'test'):

            fold = group[subject][embedding_name][fold_name]
            Y_pred = fold['Y_pred'][:]

            stimulus_ids = fold['stimulus_ids'][:]
            Y = index_unsorted(clip_features[embedding_name], stimulus_ids)

            Y, Y_pred = (torch.from_numpy(y).float() for y in (Y, Y_pred))
            Y, Y_pred = (y.flatten(start_dim=1) for y in (Y, Y_pred))
            nested_insert(results, (group_name, embedding_name), evaluate_decoding(Y, Y_pred, fold_name))

In [ ]:
# Save decoding evaluation
json_encoder = CompactJSONEncoder(indent=2)
with open('D:\\Datasets\\NSD\\derivatives\\results\\ViT-B=32\\decoding_results_01.json', 'w') as f:
    out_str = json_encoder.encode(results)
    f.write(out_str)

In [ ]:
# load

In [ ]:
group_names = ('contrastive', 'distance')
resblock_embedding_names = [f'transformer.resblocks.{i}' for i in range(12)]
embedding_names = [*resblock_embedding_names, 'embedding']
fold_name = 'val'

r2_scores = np.array(nested_select(results, [group_names, embedding_names, 'r2_score', fold_name]))
pearsonr = np.array(nested_select(results, [group_names, embedding_names, 'pearsonr', fold_name]))
contrastive_score = np.concatenate([
    np.array(nested_select(results, [group_names, resblock_embedding_names, 'mean_squared_distance', 'contrastive_score', fold_name])),
    np.array(nested_select(results, [group_names, ['embedding'], 'cosine_distance', 'contrastive_score', fold_name]))
], axis=1)

def compare_plot(data, title, x_labels, group_names):
    num_layers = data.shape[1]
    plt.plot(np.arange(num_layers), data[0], label=group_names[0])
    plt.plot(np.arange(num_layers), data[1], label=group_names[1])
    plt.xticks(np.arange(num_layers), x_labels, rotation='vertical')
    plt.title(title)
    plt.legend()
    plt.show()

short_embedding_names = [*(f'block {i}' for i in range(12)), 'embedding']
compare_plot(r2_scores, "R2 score", short_embedding_names, group_names=('contrastive loss', 'distance loss'))
compare_plot(pearsonr, "pearsonr", short_embedding_names, group_names=('contrastive loss', 'distance loss'))
compare_plot(contrastive_score, "contrastive ranking", short_embedding_names, group_names=('contrastive loss', 'distance loss'))

In [ ]:
subject = 'subj01'
embedding_name = f'transformer.resblocks.3'
results = {}

fold_name = 'val_averaged'

fold = group1[subject][embedding_name][fold_name]
Y_pred = fold['Y_pred'][:]

stimulus_ids = fold['stimulus_ids'][:]
Y = index_unsorted(clip_features[embedding_name], stimulus_ids)

Y, Y_pred = (torch.from_numpy(y).float() for y in (Y, Y_pred))
Y, Y_pred = (y.flatten(start_dim=1) for y in (Y, Y_pred))
nested_insert(results, (group_name, embedding_name), evaluate_decoding(Y, Y_pred, fold_name))

In [ ]:
# Fix incorrectly saved stimulus_ids for the non-averaged results

clip_decoded_features_path = Path('D:\\Datasets\\NSD\\derivatives\\decoded_features\\ViT-B=32')

for file_name in ('group-1.hdf5', 'group-2.hdf5'):
    with h5py.File(clip_decoded_features_path / file_name, 'a') as group:
        for embedding in group['subj01'].values():
            embedding['test']['stimulus_ids'][:] = np.repeat(embedding['test_averaged']['stimulus_ids'][:], 3)
            embedding['val']['stimulus_ids'][:] = np.repeat(embedding['val_averaged']['stimulus_ids'][:], 3)

In [ ]:
from research.data.natural_scenes import (
    NaturalScenesDataset
)

dataset_path = Path('D:\\Datasets\\NSD\\')
dataset = NaturalScenesDataset(dataset_path)

derivatives_path = dataset_path / 'derivatives'
betas_path = dataset_path / 'nsddata_betas' / 'ppdata'
ppdata_path = dataset_path / 'nsddata' / 'ppdata'
subjects = dataset.subjects

In [ ]:
encoder_name = 'fracridge'
encoder_results = h5py.File(derivatives_path / f'{encoder_name}-parameters.hdf5', 'r')

@interact(encoder_subject=encoder_results.items())
def select_subject(encoder_subject):
    subject_name = encoder_subject.name[1:]
    subject = subjects[subject_name]
    betas = subject['betas']
    roi_names = list(subject['roi_paths'].keys())
    print(roi_names)
    
    @interact(model=encoder_subject.items())
    def select_model(model, roi_name=roi_names):
        image, label_names = dataset.load_roi(subject_name, roi_name)
        image = image.T
        
        @interact(embedding=model.items())
        def select_embedding(embedding):
            
            r2 = embedding['r2'][:]

            @interact(w=(0, r2.shape[0]-1), min_r2=(0., 1.))
            def show_top(w, min_r2):
                selection_map = r2 > min_r2
                label_counts = {
                    (label_id, label_name): ((image == label_id) * selection_map).sum()
                    for label_id, label_name in label_names.items()
                }
                label_counts
                #print(label_counts)
                plt.imshow(selection_map[w].T)
                

In [ ]:
encoder_name = 'fracridge'
encoder_results = h5py.File(derivatives_path / f'{encoder_name}-parameters.hdf5', 'r')
subject_name = 'subj01'
model_name = 'ViT-B=32'
embedding_name = 'embedding'
roi_name = 'prf-visualrois'
threshold = 0.2

label_image, label_names = dataset.load_roi(subject_name, roi_name)
label_image = label_image.T

r2 = encoder_results[subject_name][model_name][embedding_name]['r2'][:]

def roi_performance(metric_map, threshold, label_image, label_names):
    selection_map = metric_map > threshold
    label_counts = {
        (label_id, label_name): ((label_image == label_id) * selection_map).sum()
        for label_id, label_name in label_names.items()
    }
    label_counts = dict(sorted(label_counts.items(), key=lambda x: x[1], reverse=True))
    return label_counts

print(roi_name, f'r2 threshold={threshold}')
roi_performance(r2, threshold, label_image, label_names)

In [ ]:
from surfer import Brain
%gui qt

In [ ]:
from surfer import Brain
from mayavi import mlab
mlab.init_notebook(backend='png')

In [ ]:
dataset_path = Path('D:\\Datasets\\NSD\\')
nsd_surfer_path = dataset_path / 'nsddata' / 'freesurfer'
subject_id = 'subj03'
hemi = 'lh'

surf_path = nsd_surfer_path / subject_id / 'surf'
surf_file_names = [p.name for p in surf_path.iterdir() if p.name.startswith(hemi)]

@interact(surf_file_name=surf_file_names)
def select_surf(surf_file_name):
    global brain
    surf = '.'.join(surf_file_name.split('.')[1:])
    print(surf)
    brain = Brain(subject_id, hemi, surf, subjects_dir=nsd_surfer_path)
    

In [ ]:
mri_path = nsd_surfer_path / subject_id / 'mri'
label_path  = nsd_surfer_path / subject_id / 'label'
overlay_file = label_path / f'{hemi}.R2.mgz'
brain.add_overlay(str(overlay_file), min=20, max=50, sign='pos')

In [ ]:
for k in brain.overlays:
    print(k)
    brain.overlays[k].remove()

In [ ]:
brain.geo[hemi].faces.shape

In [ ]:
# Init map data utility

from nsdcode.nsd_mapdata import NSDmapdata

dataset_path = Path('D:\\Datasets\\NSD\\')
derivatives_path = dataset_path / 'derivatives'
nsd = NSDmapdata(dataset_path)

In [ ]:
# Map encoding encoding model result to anat

encoder_name = 'fracridge'
model_name = 'ViT-B=32'
embedding_name = 'embedding'
image_name = 'r2'
sourcespace = 'func1pt8'
targetspace = 'anat0pt8'

targetdata = []
for i in range(8, 9):
    subject_name = f'subj0{i}'
    image_file_name = f"{'__'.join([subject_name, encoder_name, model_name, embedding_name, image_name])}.nii.gz"
    source_path = derivatives_path / 'images' / subject_name / f'{sourcespace}mm' / encoder_name 
    dest_path = derivatives_path / 'images' / subject_name / f'{targetspace}mm' / encoder_name 
    dest_path.mkdir(parents=True, exist_ok=True)
    targetdata.append(nsd.fit(
        i, 
        sourcespace=sourcespace, 
        targetspace=targetspace, 
        sourcedata=str(source_path / image_file_name),
        interptype='cubic',
        outputfile=str(dest_path / image_file_name),
    ))

In [ ]:
# Map nsdbeta r2 to anat

image_name = 'R2'
betas_name = 'betas_fithrf_GLMdenoise_RR'
sourcespace = 'func1pt8'
targetspace = 'anat0pt8'

targetdata = []
for i in range(1, 9):
    subject_name = f'subj0{i}'
    image_file_name = f"{image_name}_{betas_name}_{sourcespace}_to_{targetspace}.nii.gz"
    source_path = dataset_path / 'nsddata_betas' / 'ppdata' / subject_name
    source_path = source_path / f'{sourcespace}mm' / betas_name / f'{image_name}.nii.gz'
    dest_path = derivatives_path / 'images' / f'{targetspace}mm'
    dest_path.mkdir(parents=True, exist_ok=True)
    targetdata.append(nsd.fit(
        i, 
        sourcespace=sourcespace, 
        targetspace=targetspace, 
        sourcedata=str(source_path),
        interptype='cubic',
        outputfile=str(dest_path / image_file_name),
    ))

In [ ]:
#
#image_name = 'R2'
#betas_name = 'betas_fithrf_GLMdenoise_RR'
#sourcespace = 'func1pt8'
#def source_path(subject_name):
#    return str(dataset_path / f'nsddata_betas/ppdata/{subject_name}/{sourcespace}mm/{betas_name}/{image_name}.nii.gz')

#encoder_name = 'fracridge'
encoder_name = 'pytorch'
group_name = 'group-1'
model_name = 'ViT-B=32'
embedding_name = 'embedding'
image_name = 'r2'
sourcespace = 'func1pt8'

def source_path(subject_name):
    image_file_keys = [subject_name, encoder_name, group_name, model_name, embedding_name, image_name]
    image_file_name = f"{'__'.join(image_file_keys)}.nii.gz"
    image_folder_path = derivatives_path / 'images' / subject_name / f'{sourcespace}mm' / encoder_name 
    return str(image_folder_path / image_file_name)

hemis = ['lh', 'rh']
target_spaces = ['white', 'layerB3', 'layerB2', 'layerB1', 'pial']
subjects = [(i, f'subj0{i}') for i in range(1, 9)]

#image_file_name = f"{image_name}_{betas_name}_{sourcespace}_to_{targetspace}.nii.gz"
#dest_path = derivatives_path / 'images' / f'{targetspace}mm'
#dest_path.mkdir(parents=True, exist_ok=True)

subject_space_results = {
    subject_name: {
        hemi: {
            target_space: nsd.fit(
                subject_id, 
                sourcespace=sourcespace, 
                targetspace=f'{hemi}.{target_space}', 
                sourcedata=source_path(subject_name),
                interptype='cubic',
            )
            for target_space in target_spaces
        }
        for hemi in hemis
    }
    for subject_id, subject_name in subjects
}

fsaverage_space_results = {
    subject_name: {
        hemi: {
            target_space: nsd.fit(
                subject_id, 
                sourcespace=f'{hemi}.white', 
                targetspace='fsaverage', 
                sourcedata=subject_space_results[subject_name][hemi][target_space],
                interptype='cubic',
            )
            for target_space in target_spaces
        }
        for hemi in hemis
    }
    for subject_id, subject_name in subjects
}

In [ ]:
for subject_id, subject_name in subjects:
    for hemi in hemis:
        print({(k, v.shape) for k, v in subject_space_results[subject_name][hemi].items()})
        print({(k, v.shape) for k, v in fsaverage_space_results[subject_name][hemi].items()})

In [ ]:
from pipeline.utils import read_patch
from scipy.interpolate import griddata

lh_flat = read_patch(dataset_path / 'nsddata/freesurfer/fsaverage/surf/lh.full.flat.patch.3d')
rh_flat = read_patch(dataset_path / 'nsddata/freesurfer/fsaverage/surf/rh.full.flat.patch.3d')

print(f"{lh_flat['x'].min()=}, {lh_flat['x'].max()=}")
print(f"{lh_flat['y'].min()=}, {lh_flat['y'].max()=}")
print(f"{rh_flat['x'].min()=}, {rh_flat['x'].max()=}")
print(f"{rh_flat['y'].min()=}, {rh_flat['y'].max()=}")

def make_flat(hemi_data, hemi_flat, shape):
    hemi_data = hemi_data[hemi_flat['vno']]
    
    points = np.stack([hemi_flat['x'], hemi_flat['y']], axis=-1)
    values = hemi_data
    grid = np.stack(np.meshgrid(
        np.linspace(-160, 160, shape[0]), 
        np.linspace(-160, 160, shape[1])
    ), axis=-1).reshape(shape[0] * shape[1], 2)
    result = griddata(points, values, grid)
    return result.reshape(*shape)

@interact(subject=subjects, layer=target_spaces)
def flat_plot(subject, layer):
    global lh_data, rh_data
    
    lh_data = fsaverage_space_results[subject]['lh'][layer]
    rh_data = fsaverage_space_results[subject]['rh'][layer]
    
    lh_data = make_flat(lh_data, lh_flat, shape=(3200, 3200))
    
    print(lh_data.shape)
    

In [ ]:
lh_data

In [ ]:
@interact(subject=subjects, layer=target_spaces)
def flat_scatter_plot(subject, layer):
    #global lh_data, rh_data
    
    lh_data = fsaverage_space_results[subject]['lh'][layer]
    rh_data = fsaverage_space_results[subject]['rh'][layer]
    
    spread = 140
    y_diff = np.max(rh_flat['y']) - np.max(lh_flat['y'])
    x = np.concatenate([lh_flat['x'] - spread, rh_flat['x'] + spread])
    y = np.concatenate([lh_flat['y'], rh_flat['y'] - y_diff])
    c = np.concatenate([lh_data[lh_flat['vno']], rh_data[rh_flat['vno']]])
    
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)
    xsize = xmax - xmin
    ysize = ymax - ymin
    
    scale = 0.05
    plt.figure(figsize=(xsize * scale, ysize * scale))
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    padding=5
    fontsize=25
    plt.text(xmin+padding, ymin+padding, f'{layer=}\n{subject=}', 
             horizontalalignment='left', verticalalignment='bottom', fontsize=fontsize)
    plt.text(xmax-padding, ymin+padding, f'{image_name=}\n{model_name=}\n{embedding_name=}', 
             horizontalalignment='right', verticalalignment='bottom', fontsize=fontsize)
    plt.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    plt.scatter(x, y, s=5, c=c, cmap='jet', vmin=0.0, vmax=0.5, alpha=0.5)

In [ ]:
for subject_id, subject_name in subjects:
    for layer_name in target_spaces:
        file_name = '__'.join([subject_name, layer_name, encoder_name, model_name, embedding_name, image_name]) + '.png'
        out_path = dataset_path / f'derivatives/inspections/{subject_name}/{embedding_name}'
        out_path.mkdir(exist_ok=True, parents=True)
        flat_scatter_plot(subject_name, layer_name)
        plt.savefig(out_path / file_name, bbox_inches='tight', pad_inches=0)

In [ ]:
imageio.help(imageio.mimsave)

In [ ]:
import imageio
from glob import glob

image_paths = []
for subject_id, subject_name in subjects:
    for layer_name in target_spaces:
        path = str(dataset_path / f'derivatives/inspections/{subject_name}/embedding/{subject_name}__{layer_name}*')
        image_paths.append(glob(path)[0])
imageio.mimsave(
    dataset_path / 'derivatives/inspections/custom/all__fracridge__ViT-B=32__embedding__r2.gif', 
    [imageio.imread(im_path) for im_path in image_paths],
    fps=1,
)

image_paths = []
for subject_id, subject_name in subjects:
    path = str(dataset_path / f'derivatives/inspections/{subject_name}/embedding/{subject_name}__pial*')
    image_paths.append(glob(path)[0])
imageio.mimsave(
    dataset_path / 'derivatives/inspections/custom/pial__fracridge__ViT-B=32__embedding__r2.gif', 
    [imageio.imread(im_path) for im_path in image_paths],
    fps=0.5,
)

In [ ]:
image_paths = []
path = str(dataset_path / f'nsddata/inspections/surfacevisualizations/fsaverageflat_b*nc_groupavg.png')
image_paths = glob(path)

imageio.mimsave(
    dataset_path / 'derivatives/inspections/custom/glm_improvement.gif', 
    [imageio.imread(im_path) for im_path in image_paths],
    fps=1,
)

In [ ]:
grid = np.stack(np.meshgrid(
        np.linspace(-160, 160, shape[0]), 
        np.linspace(-160, 160, shape[1])
    )).reshape(shape[0] * shape[1], 2)
grid[3]

In [ ]:
lh_data[~np.isnan(lh_data)].max()


In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(lh_data, vmin=0, vmax=50, cmap='jet')

In [ ]:
shape=(1600, 1600)
grid = np.stack(np.meshgrid(
    np.linspace(-160, 160, shape[0]), 
    np.linspace(-160, 160, shape[1])
))
grid.shape

In [ ]:
np.linspace(

In [ ]:
# create white matter masks

spaces = ['func1pt8mm', 'func1mm']
for space in spaces:
    for i in range(1, 9):
        subject_name = f'subj0{i}'
        source_path = dataset_path / 'nsddata' / 'ppdata' / subject_name / space / 'aseg.nii.gz'
        dest_path = dataset_path / 'derivatives' / 'images' / subject_name / space
        dest_path.mkdir(parents=True, exist_ok=True)
        
        source_image = nib.load(source_path)
        source_data = source_image.get_fdata().astype(int)
        dest_data = ((source_data == 3) | (source_data == 42)) * 1
        dest_image = nib.Nifti1Image(dest_data, source_image.affine, source_image.header)
        nib.save(dest_image, dest_path / 'wm.nii.gz')


In [ ]:
# Create masks for

betas_name = 'betas_fithrf_GLMdenoise_RR'
space = 'func1pt8'

i = 1
subject_name = f'subj0{i}'
func_path = dataset_path / 'nsddata' / 'ppdata' / subject_name / f'{space}mm'
betas_path = dataset_path / 'nsddata_betas' / 'ppdata' / subject_name / f'{space}mm' / betas_name

t1_image = nib.load(func_path / f'T1_to_{space}mm.nii.gz')
r2_image = nib.load(betas_path / 'R2.nii.gz')
ncsnr_image = nib.load(betas_path / 'ncsnr.nii.gz')

r2 = np.nan_to_num(r2_image.get_fdata())
ncsnr = np.nan_to_num(ncsnr_image.get_fdata())
nc = ncsnr ** 2 / (ncsnr ** 2 + 1) * 100.
print(nc.max(), nc.min())
t1 = t1_image.get_fdata()
H, W, D = r2.shape

In [ ]:
@interact(r2_t=(0., 100.), nc_t=(0., 100.), d=(0, D-1))
def show_threshold(r2_t, nc_t, d):
    global mask
    mask = (r2 > r2_t) & (nc > nc_t)
    #print(mask.sum())
    masked_r2 = r2 * mask
    plt.figure(figsize=(10, 10))
    plt.imshow(mask[:, :, d], cmap='jet', vmin=0, vmax=1.)
#dest_path = derivatives_path / 'images' / f'{targetspace}mm'
#dest_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Save noise ceilings for voxel selection

betas_name = 'betas_fithrf_GLMdenoise_RR'
space = 'func1pt8'

def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data

with h5py.File(dataset_path / 'derivatives' / 'voxel-selection.hdf5', 'a') as f:
    for i in range(1, 9):
        subject_name = f'subj0{i}'
        func_path = dataset_path / 'nsddata' / 'ppdata' / subject_name / f'{space}mm'
        betas_path = dataset_path / 'nsddata_betas' / 'ppdata' / subject_name / f'{space}mm' / betas_name

        r2_image = nib.load(betas_path / 'R2.nii.gz')
        ncsnr_image = nib.load(betas_path / 'ncsnr.nii.gz')
        
        r2 = np.nan_to_num(r2_image.get_fdata()).astype(float).T
        ncsnr = np.nan_to_num(ncsnr_image.get_fdata()).astype(float).T
        nc = ncsnr ** 2 / (ncsnr ** 2 + 1) * 100.
        
        grid = np.argwhere(np.ones_like(r2, dtype=bool))
        
        r2_sorted_indices_flat = r2.argsort(axis=None)[::-1].astype(int)
        nc_sorted_indices_flat = nc.argsort(axis=None)[::-1].astype(int)
        r2_sorted_indices = grid[r2_sorted_indices_flat].astype(int)
        nc_sorted_indices = grid[nc_sorted_indices_flat].astype(int)
        
        group = f.require_group(subject_name)
        
        require_dataset(group, 'r2/value', r2)
        require_dataset(group, 'r2/sorted_indices_flat', r2_sorted_indices_flat)
        require_dataset(group, 'r2/sorted_indices', r2_sorted_indices)
        
        require_dataset(group, 'nc/value', nc)
        require_dataset(group, 'nc/sorted_indices_flat', nc_sorted_indices_flat)
        require_dataset(group, 'nc/sorted_indices', nc_sorted_indices)

In [ ]:
import matplotlib as mpl
fig = plt.subplots(figsize=(6, 5))
plt.hist2d(r2.flatten(), nc.flatten(), range=[[0, 100], [0, 100]], bins=(20, 20), cmap='jet', norm=mpl.colors.LogNorm())
plt.title("2D Histrogram of Voxels\nNoise Ceiling vs R^2 - Subject 1")
plt.ylabel("Noise Ceiling")
plt.xlabel("R^2")
plt.colorbar()

plt.show()

In [ ]:
nc.max()

In [ ]:
r2.flatten()